SIMP 2D
--------------------

问题目标函数定义：
$$
\begin{array}{rl} \min\limits_{\mathbf{x}}: & c(\mathbf{x})=\mathbf{U}^{\mathrm{T}}\mathbf{KU=} \displaystyle\sum\limits_{e=1}^{N} E_{e}(x_{e})\mathbf{u}_{e}^{\mathrm{T}}\mathbf{k}_{0}\mathbf{u} _{e}\\ \text{subject to}: & V(\mathbf{x)}/V_{0}=f\\ & \mathbf{K} \mathbf{U} = \mathbf{F}\\ & \mathbf{0} \le \mathbf{x} \le \mathbf{1} \end{array}
$$
使用拉格朗日乘数法进行优化，$F(\mathbf{x},\lambda) = C(x) + \lambda(V(\mathbf{x}) - c)$


### 单元（材料）刚度矩阵
KE = generate_stiffness_mat(nu)

在FEA中需要使用$k_e$生成整体刚度矩阵，需转为matrix对象返回，方便执行矩阵乘法。

$\epsilon_e = k_e u_e$

其中，$u_e$包含二维元素4个顶点的(x,y)方向位移，因此$k_e$是一个$8 \times 8$矩阵。

In [90]:
import numpy as np

def generate_stiffness_mat(nu):
    A11 = np.array([[12, 3, -6, -3], [ 3, 12,  3,  0], [-6,  3, 12, -3], [-3,  0, -3, 12]])
    A12 = np.array(np.mat('-6 -3  0  3; -3 -6 -3 -6;  0 -3 -6  3;  3 -6  3 -6'));
    B11 = np.array(np.mat('-4  3 -2  9;  3 -4 -9  4; -2 -9 -4 -3;  9  4 -3 -4'));
    B12 = np.array(np.mat('2 -3  4 -9; -3  2  9 -2;  4  9  2  3; -9 -2  3  2'));
    #KE = 1/(1-nu^2)/24*([A11 A12;A12' A11]+nu*[B11 B12;B12' B11]); 
    AA = np.vstack( (np.hstack((A11, A12)), np.hstack((A12.conj().transpose(), A11)) )) ## [A11 A12; A12' A11]
    BB = np.vstack(( np.hstack((B11, B12)), np.hstack((B12.conj().transpose(), B11)) )) ## [B11 B12; B12' B11]
    
    KE = 1/(1-nu**2)/24*(AA + nu * BB)
    return np.asmatrix(KE)

In [91]:
# test
KE = generate_stiffness_mat(0.3)
np.set_printoptions(precision=2)
print(KE)


[[ 0.49  0.18 -0.3  -0.01 -0.25 -0.18  0.05  0.01]
 [ 0.18  0.49  0.01  0.05 -0.18 -0.25 -0.01 -0.3 ]
 [-0.3   0.01  0.49 -0.18  0.05 -0.01 -0.25  0.18]
 [-0.01  0.05 -0.18  0.49  0.01 -0.3   0.18 -0.25]
 [-0.25 -0.18  0.05  0.01  0.49  0.18 -0.3  -0.01]
 [-0.18 -0.25 -0.01 -0.3   0.18  0.49  0.01  0.05]
 [ 0.05 -0.01 -0.25  0.18 -0.3   0.01  0.49 -0.18]
 [ 0.01 -0.3   0.18 -0.25 -0.01  0.05 -0.18  0.49]]


### 体结构表达

```
[iK, jK, edofMat] = generate_volume(nelx, nely)
```
2D的体结构生成:
- edofMat
包含每一行包含1个元素4个节点的自由度，每个节点2个DOF，共8个，从左下角开始，逆时针顺序排列(见论文Fig.2)。
论文中适用Matlab，从1开始索引，此处适用Python，从0开始索引。
因此通过构造每一个元素第一个自由度，通过列运算，依次得到剩余7列元素。

- iK, jK
列向量，通过Kronecker内积将节点自由度矩阵按行/列扩展8次后，展开成列向量，



In [131]:
def generate_volume(nelx, nely):    
    nodenrs = np.arange(1,(1+nelx)*(1+nely)+1).reshape(1+nely, 1+nelx, order='F').copy();
    edofVec = (2*nodenrs[0:-1, 0:-1] + 1).reshape(nelx * nely, 1, order='F');
    edofMat = np.tile(edofVec,[1,8]) + np.tile(np.hstack([np.hstack([[0,1], 2*nely + np.array([2,3,0,1])]),[-2,-1]]), [nelx*nely,1])
    edofMat -= 1   # python index start from  0 
    iK = np.kron(edofMat, np.ones([8,1])).reshape([64*nelx*nely,1])
    jK = np.kron(edofMat, np.ones([1,8])).reshape([64*nelx*nely,1])
    return iK, jK, edofMat

    

In [132]:
#test
nelx = 4
nely = 3
iK, jK, edofMat = generate_volume(nelx, nely)
print(type(edofMat))
print(edofMat)



<class 'numpy.ndarray'>
[[ 2  3 10 11  8  9  0  1]
 [ 4  5 12 13 10 11  2  3]
 [ 6  7 14 15 12 13  4  5]
 [10 11 18 19 16 17  8  9]
 [12 13 20 21 18 19 10 11]
 [14 15 22 23 20 21 12 13]
 [18 19 26 27 24 25 16 17]
 [20 21 28 29 26 27 18 19]
 [22 23 30 31 28 29 20 21]
 [26 27 34 35 32 33 24 25]
 [28 29 36 37 34 35 26 27]
 [30 31 38 39 36 37 28 29]]



### 设置边界条件

```
% DEFINE LOADS AND SUPPORTS (HALF MBB-BEAM)
F = sparse(2,1,-1,2*(nely+1)*(nelx+1),1);
U = zeros(2*(nely+1)*(nelx+1),1);
fixeddofs = union([1:2:2*(nely+1)],[2*(nelx+1)*(nely+1)]);
alldofs = [1:2*(nely+1)*(nelx+1)];
freedofs = setdiff(alldofs,fixeddofs);
```

In [152]:
from scipy import sparse
row = np.array([1])
col = np.array([0])
data = np.array([-1])
F = sparse.csc_matrix((data, (row, col)), shape=(2*(nelx+1)*(nely+1), 1)).toarray()
U = np.zeros([2*(nely+1)*(nelx+1),1])
fixeddofs = np.hstack((np.arange(1, 2*(nely+1), 2), [2*(nelx+1)*(nely+1)])) - 1
alldofs = np.arange(1, 2*(nely+1)*(nelx+1) + 1) - 1
freedofs = np.array([x for x in alldofs if x not in fixeddofs])

print(U)
print(fixeddofs)
print(alldofs.shape)
print(freedofs.shape)

[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
[ 0  2  4  6 39]
(40,)
(35,)


### FEA
用法：FEA(KE, E0, Emin, xPhys, penal, freedofs, U)

返回$(nelx+1) \times (nely+1) \times DOFs$长的列向量$U$，其中freedofs位置值为0

注意函数中：
- U使用全局变量
- Ei是$nelx \times nely$的列向量,根据密度，每个元素初始化为[0-1]之间的标准化杨氏模量 $Ei_e = x_e ^p * (E_0 - E_{min})$
- sK
 

In [153]:
import numpy.linalg as llg
def FEA(KE, E0, Emin, xPhys, penal, freedofs, U):
    Ei = np.asmatrix(xPhys.flatten() ** penal*(E0-Emin))    
    sK = np.dot(KE.flatten().transpose(), Ei).reshape(64*nely*nelx, 1, order='F')
    row = iK[:,0].transpose()
    col = jK[:,0].transpose()
    data = np.asarray(sK)[:,0].transpose()
    K = sparse.csc_matrix((data, (row,col)));
    K = (K+K.conj().T)/2;
    
    A = K[np.ix_(freedofs,freedofs)]
    B = np.asmatrix(F[freedofs])
    U[freedofs] = llg.solve(A.toarray(),B)
    return U


In [161]:
## test FEA
E0 = 1;
Emin = 1e-9;
nu = 0.3;
volfrac = 0.6
penal = 3
x = np.tile(volfrac,(nely,nelx))
xPhys = x;
U = np.zeros([2*(nely+1)*(nelx+1),1])
print(type(U))
FEA(KE, E0, Emin, xPhys, penal, freedofs, U)
print(U)
Ei = np.asmatrix(xPhys.flatten() ** penal*(E0-Emin))  
print(type(xPhys))

<class 'numpy.ndarray'>
[[  0.  ]
 [-75.52]
 [  0.  ]
 [-68.83]
 [  0.  ]
 [-65.69]
 [  0.  ]
 [-63.04]
 [-10.34]
 [-59.45]
 [ -1.28]
 [-61.75]
 [  3.45]
 [-60.79]
 [  9.96]
 [-58.72]
 [-16.65]
 [-46.22]
 [ -4.43]
 [-47.36]
 [  5.7 ]
 [-48.14]
 [ 18.08]
 [-46.92]
 [-19.73]
 [-30.31]
 [ -6.28]
 [-30.41]
 [  5.97]
 [-30.02]
 [ 24.12]
 [-30.31]
 [-20.39]
 [-15.23]
 [ -6.54]
 [-14.49]
 [  6.71]
 [-10.3 ]
 [ 28.38]
 [  0.  ]]
<class 'numpy.ndarray'>


### 生成滤波窗口
过滤器密度函数被定义为
实际算法中使用单元灵敏度$x_e$是一个局部邻域的加权平均值
$$\hat x_e=\frac{\sum_j{w{(r_{ej})x_j}}} {\sum_j w(r_{ej} )} = \sum_j{\frac{w(r_{ej})}{\sum_j w(r_{ej})}x}  = \sum_j \eta_j x_j,$$
其中，$w(r_{ej}) = max(0, r_{min}-r_{ej})$，
$r_{ej}=distant(element_e, element_j)$，
w是权重函数; 
$r_{min}$是过滤器半径。
权重因子$\eta_j$只与几何结构相关，可以事先计算，并保存固定权重因子$\eta_j$：

例如：
针对每个elment计算距离场，当rmin比较小时，这是一个稀疏矩阵：
$$
H = \begin{bmatrix}
 0 &  0            & 1     & 2      & \dots & nelx \times nely \\    
 0 & d_{00}       & d_{01} & x_{02} & \dots & d_{0n} \\
 1 & d_{10}       & d_{11} & x_{12} & \dots & d_{1n} \\
 \dots \\
 nelx \times nely &  d_{y0}       & d_{y1} & d_{y3} & \dots & d_{yn}
\end{bmatrix}
$$

$$Hs = 
\begin{bmatrix} 
\sum_j w(r_{ej})_0 \\
\dots \\
\sum_j w(r_{ej})_{nely * nelx - 1}
\end{bmatrix}
$$
H, Hs = preFilter(rmin)

In [136]:
def preFilter(rmin):
    nTotal = nelx*nely*(2*(np.ceil(rmin)-1)+1)**2    # nTotal = x*y*Neighbors_per_element
    iH = np.ones(int(nTotal))
    jH = np.ones(iH.shape)
    sH = np.zeros(iH.shape)
    
    k = 0
    for i in np.arange(0,nelx):
        for j in np.arange(0,nely):
            e = i*nely + j
            neighbor_size = np.ceil(rmin) - 1 # unit: element
            neighbor_x = np.arange(max(i-neighbor_size, 0), min(i+neighbor_size+1, nelx))
            neighbor_y = np.arange(max(j-neighbor_size, 0), min(j+neighbor_size+1, nely))
            for ni in neighbor_x:
                for nj in neighbor_y:
                    neighbor_e = ni * nely + nj
                    iH[k] = e
                    jH[k] = neighbor_e
                    sH[k] = max(0, rmin - np.sqrt((ni-i)**2 +(nj-j)**2 ))
                    k += 1
    
    H = sparse.csc_matrix((sH, (iH, jH)) )
    Hs = np.sum(H, axis=1)
    return H, Hs

In [137]:
# test filter
rmin = 1.2
H, Hs = preFilter(rmin)
print(H.todense())
print(Hs)


[[ 1.2  0.2  0.   0.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.2  1.2  0.2  0.   0.2  0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.2  1.2  0.   0.   0.2  0.   0.   0.   0.   0.   0. ]
 [ 0.2  0.   0.   1.2  0.2  0.   0.2  0.   0.   0.   0.   0. ]
 [ 0.   0.2  0.   0.2  1.2  0.2  0.   0.2  0.   0.   0.   0. ]
 [ 0.   0.   0.2  0.   0.2  1.2  0.   0.   0.2  0.   0.   0. ]
 [ 0.   0.   0.   0.2  0.   0.   1.2  0.2  0.   0.2  0.   0. ]
 [ 0.   0.   0.   0.   0.2  0.   0.2  1.2  0.2  0.   0.2  0. ]
 [ 0.   0.   0.   0.   0.   0.2  0.   0.2  1.2  0.   0.   0.2]
 [ 0.   0.   0.   0.   0.   0.   0.2  0.   0.   1.2  0.2  0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.2  0.   0.2  1.2  0.2]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0.2  0.   0.2  1.2]]
[[ 1.6]
 [ 1.8]
 [ 1.6]
 [ 1.8]
 [ 2. ]
 [ 1.8]
 [ 1.8]
 [ 2. ]
 [ 1.8]
 [ 1.6]
 [ 1.8]
 [ 1.6]]


### 柔度和灵敏度分析
- 计算每个单元的柔度$c_e$
$$c_e=u_e^Tk_e  u_e$$

  这里$u_e$是一个8$\times$1的列向量，在上面代码中，将其转为行向量，对于所有N个元素，构造了一个N$\times$8的矩阵$\mathbf{U}$，$\mathbf U *k_e.* U^T$，然后按行求和，得到的$c_e$是一个N维列向量，每个元素代表一个单元的整体变性能。再将其转为一个nelx $\times$ nely的矩阵，与结构相对应。
  

- 计算整体柔度，对每一个元素的乘以相应的密度，然后对所有元素求和。在实际代码中，每个元素的密度首先根据SIMP的材料模型$E(x_e) = E_0x_e^p$来计算[Huang & Xie 2009](http://t.cn/RarnNu1)，得到的结果再进行滤波，以模拟物理密度。
$$Ce =U^T K U = \sum_{e=1}^N E_e(x_e)c_e = \sum_{e=1}^N (E_{min} + x_e^p(E_0 - E_{min}))c_e$$

- 敏感度的计算就仍然采用SIMP材料模型，柔度对单元密度进行求导：
  $\alpha_e= \frac{\partial(\mathcal{C})}{\partial(\mathcal{x}_e)}$

  $\alpha_e = -p\mathcal{x}_e^{p-1} (E_0 - E_{min})\mathbf{u}_e^T\mathbf{k}_0\mathbf{u}_e = -p\mathcal{x}_e^{p-1} (E_0 - E_{min}) c_e$

  其中，可变量为$-p\mathcal{x}_e^{p-1}$,因为在单相材料的优化问题中，SIMP使用了差值方法使材料在数学上保持近似连续分布，此时杨氏模量$E(x_e)= E_{solid}x_e^p$，$p$为惩罚因子，可以保证$0 < E(x_e) < E_{solid}$。
  
  从上边的计算中可以看到，灵敏度中$E_e = x^p \mathbf{u}_e^Tk_0 \mathbf{u}_e$为实体单元应变能，也可以从可直接从FEA中获得。
  
  本例中使用密度矩阵和柔度矩阵直接得到计算敏感度矩阵dc，然后进行滤波：
  
  $\frac{\hat{\partial c}}{\partial x_e} = \sum_{i \in N} \frac{w_{e,i}}{\sum_{i \in N} w_{e,i}}  x_e \frac{\partial c} {\partial x_e}$
  
   其中$w_{e,i}$已经保存在H的行中，$\sum_{i \in N} w_{e,i}$存储在$H_s$中。



In [138]:
a = np.multiply(np.asmatrix(U[edofMat]) * (KE), np.asmatrix(U[edofMat]))
ce = np.sum(a, axis=1).reshape(nely, nelx, order='F')
c = np.sum(np.sum(np.multiply((Emin+xPhys**penal *(E0-Emin)), ce)))
dc = np.multiply(-penal * (E0-Emin)*xPhys**(penal-1),ce)
dv = np.ones((nely,nelx))
np.set_printoptions(precision=2)
print(dc)
#print(dc.flatten(order='F').T)
print(H * np.multiply(x, dc).flatten(order='F').T)
# Filtering dc
#dc[:] = np.multiply(H * np.asmatrix(x).flatten().transpose(), dc.flatten().transpose())#/ Hs
#filering dc
dc =( H * np.multiply(x, dc).flatten(order='F').T / Hs ).reshape(nely, nelx, order='F') 
print(dc)



[[-97.19 -29.96 -12.32  -2.02]
 [-13.91 -19.99 -18.17 -18.66]
 [-55.14 -36.97 -17.58 -55.7 ]]
[[-75.24]
 [-30.7 ]
 [-45.81]
 [-37.11]
 [-26.27]
 [-37.74]
 [-14.89]
 [-21.31]
 [-25.96]
 [ -5.17]
 [-22.54]
 [-44.45]]
[[-47.03 -20.62  -8.27  -3.23]
 [-17.05 -13.14 -10.65 -12.52]
 [-28.63 -20.97 -14.42 -27.78]]


### 优化迭代
 
- 更新策略
 
  选择简单优化准则方法（Optimality criteria，OC）：
  $$
    x_{e}^{\mathrm{new}} =
    \begin{cases} 
      \max(0,x_{e} - m) & \text{if $x_{e} B^{\eta}_{e} \le \max(0,x_{e} - m)$} \\ 
      \min(1,x_{e} + m) & \text{if x_{e} B^{\eta}_{e} \ge \min(1,x_{e} - m)} \\ 
       x_{e} B^{\eta}_{e} & \text{otherwise} 
    \end{cases},
  $$ 
  其中$m$是更新步长。这个更新策略的目标简单来说是
    Volume = sum(sum(x))是一个关于拉格朗日乘数($\lambda$)的单调递减函数，
    $$$$
    一般bi-sectioning算法选择合适的$\lambda$。逐步缩小
     
     - ;


In [164]:
def OC(nelx, nely, xPhys, volfrac, dc, ft):
    l1 = 0
    l2 = 100000 
    move = 0.2
    while (l2-l1)/(l1+l2) > 1e-3:
        lmid = 0.5*(l2+l1);
        xnew = np.maximum(0,np.maximum(x-move,np.minimum(1,np.minimum(x+move,np.multiply(x, np.sqrt(-dc/dv/lmid))))));
        
        if ft == 1:
            xPhys = np.asarray(xnew)
        else:
            if ft == 2:
               xPhys = (H * xnew.flatten().T) / Hs;
    
        if np.sum(xPhys.flatten()) > volfrac*nelx*nely:
           l1 = lmid
        else:
           l2 = lmid
    return xPhys

In [171]:
#test OC and bisection algorithm
l1 = 0
l2 = 100000 
move = 0.2
ft = 1
lmid = 0.5*(l2+l1)

print(xPhys)
#a = np.multiply(x, np.sqrt(-dc/dv/lmid))
xPhys = OC(nelx, nely, xPhys, volfrac, dc, ft)
#OC(nelx, nely, xPhys, volfrac, dc, 1)
print(xPhys)
type(xPhys)#.flatten() ** penal*(E0-Emin))
#type(np.asarray(xPhys))
#Ei = np.asmatrix(xPhys.flatten() ** penal*(E0-Emin))

[[ 0.6  0.6  0.6  0.6]
 [ 0.6  0.6  0.6  0.6]
 [ 0.6  0.6  0.6  0.6]]
[[ 0.8   0.66  0.42  0.4 ]
 [ 0.6   0.53  0.48  0.52]
 [ 0.78  0.67  0.56  0.77]]


numpy.ndarray

### Optimization Iteration



In [173]:
# optimization

change = 1

loop = 0
Max_Loop = 20

E0 = 1;
Emin = 1e-9;
nu = 0.3;
volfrac = 0.6
penal = 3
x = np.tile(volfrac,(nely,nelx))
xPhys = x;
U = np.zeros([2*(nely+1)*(nelx+1),1])
while change > 0.01 and loop < Max_Loop:
        loop += 1
        FEA(KE, E0, Emin, xPhys, penal, freedofs, U)
        ## Objective function and sensitivity analysis
        print(loop)
        a = np.multiply(np.asmatrix(U[edofMat]) * (KE), np.asmatrix(U[edofMat]))
        ce = np.sum(a, axis=1).reshape(nely, nelx, order='F')
        c = np.sum(np.sum(np.multiply((Emin+xPhys**penal *(E0-Emin)), ce)))
        dc = np.multiply(-penal * (E0-Emin)*xPhys**(penal-1),ce)
        dv = np.ones((nely,nelx))
        
        ## Filtering and modificating the sensitive
        if ft == 1:
            b = np.max(x)
            print(b)
            dc =( H * np.multiply(x, dc).flatten(order='F').T / Hs / b ).reshape(nely, nelx, order='F') 
    
            
        ## Update
        xPhys = OC(nelx, nely, xPhys, volfrac, dc, ft)
        change = np.max(np.abs(xPhys-x));
        x = xPhys
        
        print("loop: " + str(loop))
        print(x)
        print(change)
print('End')

1
0.6
loop: 1
[[ 0.8   0.66  0.42  0.4 ]
 [ 0.6   0.53  0.48  0.52]
 [ 0.78  0.67  0.56  0.77]]
0.2
2
0.8
loop: 2
[[ 1.    0.86  0.41  0.2 ]
 [ 0.75  0.6   0.28  0.32]
 [ 0.98  0.87  0.36  0.57]]
0.2
3
1.0
loop: 3
[[ 1.    1.    0.61  0.05]
 [ 0.87  0.8   0.13  0.14]
 [ 1.    1.    0.2   0.39]]
0.2
4
1.0
loop: 4
[[ 1.    1.    0.81  0.04]
 [ 0.67  1.    0.1   0.08]
 [ 1.    0.91  0.12  0.48]]
0.2
5
1.0
loop: 5
[[ 1.    1.    1.    0.04]
 [ 0.49  1.    0.11  0.04]
 [ 0.97  0.84  0.07  0.63]]
0.1889675362
6
1.0
loop: 6
[[ 1.    1.    1.    0.02]
 [ 0.54  1.    0.13  0.02]
 [ 0.82  1.    0.03  0.64]]
0.1556035609
7
1.0
loop: 7
[[ 1.    1.    1.    0.  ]
 [ 0.61  1.    0.14  0.01]
 [ 0.67  1.    0.02  0.75]]
0.151149763245
8
1.0
loop: 8
[[  1.00e+00   1.00e+00   1.00e+00   9.27e-05]
 [  6.84e-01   1.00e+00   1.61e-01   3.71e-03]
 [  5.06e-01   1.00e+00   1.65e-02   8.30e-01]]
0.159704858725
9
1.0
loop: 9
[[  1.00e+00   1.00e+00   1.00e+00   1.77e-06]
 [  6.92e-01   1.00e+00   2.80e-01   2.